In [ ]:
# ===============================================================================================================#
# Copyright 2023 Infosys Ltd.                                                                          #
# Use of this source code is governed by Apache License Version 2.0 that can be found in the LICENSE file or at  #
# http://www.apache.org/licenses/                                                                                #
# ===============================================================================================================#

## UC_40 - Retrieval from vectordb
<div  style="line-height: 1;">
    <span style="color:Green">After running indexing pipeline run this pipeline.</span></div>

#### Import libraries

In [ ]:
import os
import json
import shutil
import infy_dpp_sdk
import infy_fs_utils
from _internal_utils.pipeline_helper import PipelineHelper
import warnings
warnings.simplefilter("ignore")

#### Set environment variables
<div  style="line-height: 1;">
    <span style="color:Red"><b>NOTE:</b> The Pipeline uses environment variables which needs to be set by the developer.<br>
In production developer needs to set them as required.<br>
In this notebook you can provide them using the below code.<br>
To set or change the value please refer <i>installation.ipynb</i></span>
</div>

In [ ]:
%store -r USE_LOCAL_STORE
if 'USE_LOCAL_STORE' in locals() and USE_LOCAL_STORE:
    %store -r AZURE_OPENAI_SERVER_BASE_URL
    os.environ['AZURE_OPENAI_SERVER_BASE_URL']=AZURE_OPENAI_SERVER_BASE_URL
    %store -r AZURE_OPENAI_SECRET_KEY
    os.environ['AZURE_OPENAI_SECRET_KEY']=AZURE_OPENAI_SECRET_KEY
    %store -r LITELLM_PROXY_SERVER_BASE_URL
    os.environ['LITELLM_PROXY_SERVER_BASE_URL']=LITELLM_PROXY_SERVER_BASE_URL
    %store -r LITELLM_PROXY_SECRET_KEY
    os.environ['LITELLM_PROXY_SECRET_KEY']=LITELLM_PROXY_SECRET_KEY
    %store -r INFY_DB_SERVICE_BASE_URL
    os.environ['INFY_DB_SERVICE_BASE_URL']=INFY_DB_SERVICE_BASE_URL
    %store -r INFY_MODEL_SERVICE_BASE_URL
    os.environ['INFY_MODEL_SERVICE_BASE_URL']=INFY_MODEL_SERVICE_BASE_URL
    %store -r INFY_RESOURCE_SERVICE_BASE_URL
    os.environ['INFY_RESOURCE_SERVICE_BASE_URL']=INFY_RESOURCE_SERVICE_BASE_URL
    %store -r INFY_SEARCH_SERVICE_BASE_URL
    os.environ['INFY_SEARCH_SERVICE_BASE_URL']=INFY_SEARCH_SERVICE_BASE_URL
else:
    print("USE_LOCAL_STORE is not set to true. DPP pipeline will use system environment variables.")

#### Define configuration file path

In [ ]:
STORAGE_ROOT_PATH = 'C:/del/fs/notebookuc/STORAGE'
CONTAINER_ROOT_PATH = 'C:/del/fs/notebookuc/CONTAINER'
PIPELINE_INPUT_CONFIG_FILE_PATH = '/data/config/dpp_pipeline_retriever_input_config.json'

#### Copying files
<div  style="line-height: 1;"><span style="color:Red"><b>NOTE: </b>In this notebook below is used to copy sample files to folders in <i>STORAGE_ROOT_PATH</i>.<br>
In production the data and config files should kept under respective folders in <i>STORAGE_ROOT_PATH </i>.<br>
<span></div>

In [ ]:
current_data_path = os.path.abspath('./data')

if not os.path.exists(f'{STORAGE_ROOT_PATH}/data'):
    os.makedirs(f'{STORAGE_ROOT_PATH}/data')
if not os.path.exists(f'{STORAGE_ROOT_PATH}/data/input'):
    os.makedirs(f'{STORAGE_ROOT_PATH}/data/input')     
shutil.copytree(f'{current_data_path}/sample/config',f'{STORAGE_ROOT_PATH}/data/config',
                dirs_exist_ok=True)

In [ ]:
# Show pipeline card
PipelineHelper(PIPELINE_INPUT_CONFIG_FILE_PATH, STORAGE_ROOT_PATH, CONTAINER_ROOT_PATH).show_pipeline_card()

#### Initialize Client Config

In [ ]:
storage_config_data = infy_fs_utils.data.StorageConfigData(
        **{
            "storage_root_uri": f"file://{STORAGE_ROOT_PATH}",
            "storage_server_url": "",
            "storage_access_key": "",
            "storage_secret_key": ""
        })

file_sys_handler = infy_fs_utils.provider.FileSystemHandler(
    storage_config_data)
if not infy_fs_utils.manager.FileSystemManager().has_fs_handler(
    infy_dpp_sdk.common.Constants.FSH_DPP):
    infy_fs_utils.manager.FileSystemManager().set_root_handler_name(
                    infy_dpp_sdk.common.Constants.FSH_DPP)
    infy_fs_utils.manager.FileSystemManager().add_fs_handler(file_sys_handler)

In [ ]:
# Configure client properties
client_config_data = infy_dpp_sdk.ClientConfigData(
    **{
        "container_data": {
            "container_root_path": f"{CONTAINER_ROOT_PATH}",
        }
    })
infy_dpp_sdk.ClientConfigManager().load(client_config_data)

#### Initialize Logging

In [ ]:
import logging
# Modify as required to control the overall logging level
logging.basicConfig(level=logging.ERROR)
logging_config_data = infy_fs_utils.data.LoggingConfigData(
        **{
            # "logger_group_name": "my_group_1",
            "logging_level": 40,
            "logging_format": "",
            "logging_timestamp_format": "",
            "log_file_data": {
                "log_file_dir_path": "/logs",
                "log_file_name_prefix": "retriever_batch",
                # "log_file_name_suffix": "1",
                "log_file_extension": ".log"

            }})

In [ ]:
if not infy_fs_utils.manager.FileSystemLoggingManager().has_fs_logging_handler(
    infy_dpp_sdk.common.Constants.FSLH_DPP):
    file_sys_logging_handler = infy_fs_utils.provider.FileSystemLoggingHandler(
                logging_config_data, file_sys_handler)
    infy_fs_utils.manager.FileSystemLoggingManager(
            ).set_root_handler_name(infy_dpp_sdk.common.Constants.FSLH_DPP)
    infy_fs_utils.manager.FileSystemLoggingManager(
            ).add_fs_logging_handler(file_sys_logging_handler)

### Run the pipeline

In [ ]:
dpp_orchestrator = infy_dpp_sdk.orchestrator.OrchestratorNative(
        input_config_file_path=PIPELINE_INPUT_CONFIG_FILE_PATH)

In [ ]:
processor_response_list = dpp_orchestrator.run_batch()

### Verify results

<div  style="line-height: 1;">
    <span style="color:Green"><b>NOTE: </b> The results of the pipeline will be available in <i>processor_response_data.json</i> file at <i>work_folder_path</i>.</span></div>

In [ ]:
print(json.dumps(processor_response_list[0].
                 context_data.get('request_closer'),indent=4))

<div  style="line-height: 1;">
    <span style="color:Green"><b>NOTE:</b> For the provided query based on similarity top matches are show below.</span></div>

In [ ]:
print(json.dumps(processor_response_list[0].context_data.get("query_retriever"),indent=4))

<div style="line-height: 1;">
<span style="color:Red"><b>NOTE: </b>for rerunning the inference pipeline, please move the request file (<b>the 'group_request_file' from uc_30_indexing_vectordb.ipynb</b> ) from <i>work/request/inference/complete</i> to <i>work/request/indexer/complete</i>.<br>
</span>
</div>